In [7]:

import os
os.environ["OMP_NUM_THREADS"] = "1"

import pandas as pd
import numpy as np
import scipy.io
import scanpy as sc
import argparse
from scipy import sparse
from tqdm import tqdm

# CellSNP de novo genotyping 

## Multi-sample pipeline

### STEP1

In [ ]:
%%bash

BAMS="/data02/zhaogefei/pbmc_public/GSE163160/pbmc1/SRR13252434_atac/outs/possorted_bam.bam,\
/data02/zhaogefei/pbmc_public/GSE163160/pbmc2/SRR13252435_atac/outs/possorted_bam.bam,\
/data02/zhaogefei/pbmc_public/GSE163160/pbmc3/SRR13252436_atac/outs/possorted_bam.bam,\
/data02/zhaogefei/pbmc_public/GSE163160/pbmc4/SRR13252437_atac/outs/possorted_bam.bam,\
/data02/zhaogefei/pbmc_public/GSE163160/pbmc5/SRR13252438_atac/outs/possorted_bam.bam,\
/data02/zhaogefei/pbmc_public/GSE163160/pbmc6/SRR13252439_atac/outs/possorted_bam.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/1kng11/1kng11.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/5kng11/5kng11.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/10kng11/10kng11.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/10kv2cc/10kv2cc.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/10kv2cx/10kv2cx.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/10kv11cx/10kv11cx.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/10kv11cxm/10kv11cxm.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/500ng11/500ng11.bam,\
/data02/zhaogefei/pbmc_public/10Xpublic/10kv11cc/10k_PBMC_ATAC_nextgem_Chromium_Controller_possorted_bam.bam"


OUT_DIR="~/denovo_discovery"
mkdir -p ${OUT_DIR}

echo "Starting De Novo SNP Discovery on ALL BAMs..."


# --minMAF 0.05:  min counting allele frequency 
# --minCOUNT 100: min counting reads supporting SNP
# --chrom:  keep only standard chromosomes

cellsnp-lite \
    -s ${BAMS} \
    -O ${OUT_DIR} \
    --minMAF 0.05 \
    --minCOUNT 100 \
    --chrom chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX,chrY \
    --cellTAG None \
    --UMItag None \
    -p 24 \
    --genotype \
    --gzip

echo "Done! The dictionary file is located at: ${OUT_DIR}/cellSNP.base.vcf.gz"

### STEP2

In [ ]:
import gzip
import sys
import os


DENOVO_FILE = "~/denovo_discovery/cellSNP.base.vcf.gz"
# Reference dbSBP or 1KG VCF
REF_FILE = "broad_hg38.vcf.gz"

OUT_FILE = "~/denovo_discovery/cellSNP.filtered_1kG.vcf.gz"
# ===========================================

def filter_vcf():
    print(f"Start filtering...")
    print(f"Input: {DENOVO_FILE}")
    print(f"Reference: {REF_FILE}")

    ref_sites = set()
    print("Loading Reference positions into memory...")
    
    try:
        with gzip.open(REF_FILE, 'rt') as f:
            for line in f:
                if line.startswith("#"): continue
                parts = line.split('\t', 2)
            
                ref_sites.add(f"{parts[0]}:{parts[1]}")
    except Exception as e:
        print(f"Error reading reference: {e}")
        return

    print(f"Loaded {len(ref_sites)} sites from Reference.")


    print("Filtering De Novo variants...")
    kept_count = 0
    total_count = 0
    
    try:
        with gzip.open(DENOVO_FILE, 'rt') as fin, gzip.open(OUT_FILE, 'wt') as fout:
            for line in fin:
              
                if line.startswith("#"):
                    fout.write(line)
                    continue
                total_count += 1
                parts = line.split('\t', 2)
                key = f"{parts[0]}:{parts[1]}"
            
                if key in ref_sites:
                    fout.write(line)
                    kept_count += 1

                if total_count % 100000 == 0:
                    print(f"Processed {total_count} lines, kept {kept_count}...")

    except Exception as e:
        print(f"Error processing de novo file: {e}")
        return

    print("="*30)
    print("Done!")
    print(f"Total De Novo variants: {total_count}")
    print(f"Variants kept (in 1kG): {kept_count}")
    print(f"Output saved to: {OUT_FILE}")

if __name__ == "__main__":
    filter_vcf()

#RUN BASH SCRIPT TO EXECUTE THE PIPELINE

### STEP3

In [ ]:
%%bash

# The saame as first demonstration but with barcode files added
# Format: SAMPLE  BAM FILE  CELLTAG  BARCODE FILE
BAM_LIST="bam_list.tsv" 
OUT_BASE="~/cellsnp_pre_sampler"
VCF_FILE="~/denovo_discovery/cellSNP.filtered_1kG.vcf.gz"

mkdir -p ${OUT_BASE}

echo "=========================================="
echo "Starting cellsnp-lite processing"
echo "BAM list: ${BAM_LIST}"
echo "Output base: ${OUT_BASE}"
echo "VCF file: ${VCF_FILE}"
echo "=========================================="


while read SAMPLE BAM CELLTAG BARCODE_FILE; do
    echo ">>> Processing ${SAMPLE} (${CELLTAG})"
    echo "   BAM: ${BAM}"
    echo "   Barcode file: ${BARCODE_FILE}"

    if [ ! -f "${BAM}" ]; then
        echo "   ERROR: BAM file not found: ${BAM}"
        echo "   Skipping ${SAMPLE}"
        continue
    fi
    
    if [ ! -f "${BARCODE_FILE}" ]; then
        echo "   ERROR: Barcode file not found: ${BARCODE_FILE}"
        echo "   Skipping ${SAMPLE}"
        continue
    fi
    
    BARCODE_COUNT=$(wc -l < "${BARCODE_FILE}" 2>/dev/null || echo 0)
    if [ "${BARCODE_COUNT}" -eq 0 ]; then
        echo "   WARNING: Barcode file is empty: ${BARCODE_FILE}"
        echo "   Skipping ${SAMPLE}"
        continue
    fi
    
    OUT_DIR=${OUT_BASE}/${SAMPLE}
    mkdir -p ${OUT_DIR}
    
    echo "   Barcodes: ${BARCODE_COUNT}"
    echo "   Output dir: ${OUT_DIR}"
    echo "   Running cellsnp-lite..."
    
    cellsnp-lite \
        -s ${BAM} \
        -b ${BARCODE_FILE} \
        -O ${OUT_DIR} \
        -R ${VCF_FILE} \
        --cellTAG ${CELLTAG} \
        --UMItag None \
        --minMAF 0 \
        --minCOUNT 0 \
        --chrom chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22 \
        -p 20 \
        --genotype 

    
done < ${BAM_LIST}

# Vireo For Donors discoverey

In [ ]:
import os
import scipy.io as sio
from scipy.sparse import hstack
import numpy as np
#import gzip
import shutil

BASE_DIR = "~/cellsnp_pre_sampler"


OUT_DIR = "~/cellsnp_pre_sampler/merged_output"

# BASE_DIR/sample0, BASE_DIR/sample1
SAMPLES = ['SRR13252434',
 'SRR13252435',
 'SRR13252436',
 'SRR13252437',
 'SRR13252438',
 'SRR13252439',
 '1kng11',
 '5kng11',
 '10kng11',
 '10kv2cc',
 '10kv2cx',
 '10kv11cx',
 '10kv11cxm',
 '500ng11',
 '10kv11cc',
]
# ===========================================

def merge_cellsnp_outputs():
    
    if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)
        print(f"Created output directory: {OUT_DIR}")

    ad_matrices = []
    dp_matrices = []
    all_barcodes = []
    
    n_rows_check = None



    for sample in SAMPLES:
        sample_path = os.path.join(BASE_DIR, sample)
        print(f"Processing: {sample} ...")

    
        try:
            ad = sio.mmread(os.path.join(sample_path, "cellSNP.tag.AD.mtx")).tocsr()
            dp = sio.mmread(os.path.join(sample_path, "cellSNP.tag.DP.mtx")).tocsr()
        except FileNotFoundError:
            print(f"Error: No {sample} .mtx files, PASS")
            continue

       
        if n_rows_check is None:
            n_rows_check = ad.shape[0]
        else:
            if ad.shape[0] != n_rows_check:
                raise ValueError(
                    f"Should have same SNP rows!"
                )

   
        bc_file = os.path.join(sample_path, "cellSNP.samples.tsv")
        with open(bc_file, 'r') as f:
            bcs = [line.strip() for line in f]
        
        bcs_modified = [f"{sample}_{bc}" for bc in bcs]

       
        if len(bcs_modified) != ad.shape[1]:
            raise ValueError(f"Sample {sample} Barcodes numbers not match AD matrix columns!")

    
        ad_matrices.append(ad)
        dp_matrices.append(dp)
        all_barcodes.extend(bcs_modified)

    print("-" * 30)
    print(f"SANPLES: {len(ad_matrices)}")
    print(f"SNPs per sample (Rows): {n_rows_check}")
    print(f"BARCODES (Cols): {len(all_barcodes)}")
    

    print("Horizontal Stack...")
    merged_ad = hstack(ad_matrices)
    merged_dp = hstack(dp_matrices)


    print(f"TO {OUT_DIR} ...")
    sio.mmwrite(os.path.join(OUT_DIR, "cellSNP.tag.AD.mtx"), merged_ad)
    sio.mmwrite(os.path.join(OUT_DIR, "cellSNP.tag.DP.mtx"), merged_dp)
    with open(os.path.join(OUT_DIR, "cellSNP.samples.tsv"), 'w') as f:
        for bc in all_barcodes:
            f.write(bc + "\n")

    first_vcf = os.path.join(BASE_DIR, SAMPLES[0], "cellSNP.base.vcf")
    target_vcf = os.path.join(OUT_DIR, "cellSNP.base.vcf")
    
    if os.path.exists(first_vcf):
        shutil.copy(first_vcf, target_vcf)
        print("WELL DONE: VCF file copied to output directory.")
    else:
        print("NO VCF file found to copy.")

if __name__ == "__main__":
    merge_cellsnp_outputs()

In [ ]:
%%bash

CELLSNP_DIR="~/cellsnp_pre_sampler/merged_output"


OUT_DIR="~/Vireo"
mkdir -p $OUT_DIR

# -c: cellsnp-lite output dir (AFTER MERGE)
# -N: how many samples to deconvolute (DONOR NUMBER)
# -o: out dir
# -p: task per CPU

vireo -c $CELLSNP_DIR -N 8 -o $OUT_DIR -p 20

# Indentity cancidate ASA

### ON SUB CELLTYPE

In [ ]:
adata=sc.read_h5ad('CD4_T_subfilter.h5ad')

#### Donor Aggregation
Strict Reads Imbalance on SNV positon (Check snv position coverage)


In [31]:
vireo_df=pd.read_csv("~/donor_ids.tsv", sep="\t", index_col=0)
if isinstance(adata.X, np.ndarray):
    counts = adata.X.sum(axis=1)
else:
    counts = adata.X.sum(axis=1).A1  

adata.obs['total_fragments'] = counts

common_cells = adata.obs_names.intersection(vireo_df.index)
meta_df = pd.DataFrame(index=common_cells)
meta_df['donor_id'] = vireo_df.loc[common_cells, 'donor_id']
meta_df['total_fragments'] = adata.obs.loc[common_cells, 'total_fragments']
meta_df = meta_df[~meta_df['donor_id'].isin(['unassigned', 'doublet'])]
meta_df.to_csv("cell_meta.tsv", sep="\t", index_label="cell")


In [ ]:

#meta_path = "cell_meta.tsv"
#meta_df = pd.read_csv(meta_path, sep="\t", index_col=0)
TARGET_CELL_TYPE = "cd4" 


target_barcodes = meta_df.index.values

# ==========================================
print("Loading VCF...")
vcf_df = pd.read_csv("~/cellsnp_pre_sampler/merged_output/cellSNP.base.vcf.gz", 
                     sep="\t", comment='#', header=None, compression='gzip')
snp_ids = (vcf_df[0].astype(str) + "_" + vcf_df[1].astype(str) + "_" + 
           vcf_df[3].astype(str) + "_" + vcf_df[4].astype(str)).values


print("Loading Matrices...")
ad_mtx = sio.mmread("~/cellsnp_pre_sampler/merged_output/cellSNP.tag.AD.mtx").tocsr()
dp_mtx = sio.mmread("~/cellsnp_pre_sampler/merged_output/cellSNP.tag.DP.mtx").tocsr()
matrix_barcodes = pd.read_csv("~/cellsnp_pre_sampler/merged_output/cellSNP.samples.tsv", header=None)[0].values

# SLICE
print("Slicing matrix for target cells...")
_, matrix_idx, target_idx = np.intersect1d(matrix_barcodes, target_barcodes, return_indices=True)

if len(matrix_idx) == 0:
    raise ValueError("ERROR:CHECK BARCODE")

ad_mtx_sub = ad_mtx[:, matrix_idx]
dp_mtx_sub = dp_mtx[:, matrix_idx]
barcode_to_donor = meta_df['donor_id'].to_dict()
sub_donors = np.array([barcode_to_donor[bc] for bc in matrix_barcodes[matrix_idx]])

# ==========================================

results = []
unique_donors = np.unique(sub_donors)

for snp_idx in tqdm(range(ad_mtx_sub.shape[0])):
    row_dp = dp_mtx_sub[snp_idx, :].toarray().flatten()
    if row_dp.sum() == 0: continue
    
    row_ad = ad_mtx_sub[snp_idx, :].toarray().flatten()
    current_snp_id = snp_ids[snp_idx] 
    
    for donor in unique_donors:
        donor_mask = (sub_donors == donor)
        valid_mask = donor_mask & (row_dp > 0)
        
        if not np.any(valid_mask): continue
        
        count_pos = row_ad[valid_mask].sum()
        count_total = row_dp[valid_mask].sum()
        count_neg = count_total - count_pos
        n_cells = np.sum(valid_mask)
        
        if count_total < 10 or n_cells < 5: continue
        
        results.append({
            "SNP": current_snp_id,
            "CellType": TARGET_CELL_TYPE,
            "Donor": donor,
            "Count_Pos": count_pos,
            "Count_Neg": count_neg,
            "N_Cells": n_cells
        })

df_final = pd.DataFrame(results)

#print(df_final.head())
#              SNP CellType   Donor  Count_Pos  Count_Neg  N_Cells
#  chr1_10231_C_A      cd4  donor5          5          5       10
#  chr1_10291_C_T      cd4  donor4          0         11       10
#  chr1_10291_C_T      cd4  donor5          5         37       38
#  chr1_10327_T_C      cd4  donor4          3         10       12
#  chr1_10327_T_C      cd4  donor5          1         16       15


In [ ]:
peak_names = adata.var_names 


peak_data = []
for p in peak_names:
    parts = p.replace(":", "-").split("-")
    peak_data.append({
        "chrom": parts[0],
        "start": int(parts[1]),
        "end": int(parts[2]),
        "peak_id": p  
    })

df_peaks = pd.DataFrame(peak_data)

print("Mapping SNPs to Peaks (Robust Method)...")


snp_meta = df_final['SNP'].str.split('_', expand=True)
df_final['chrom'] = snp_meta[0]
df_final['pos'] = snp_meta[1].astype(int)
peak_data = []
for p in peak_names:
    parts = p.replace(":", "-").split("-")
    peak_data.append({
        "chrom": parts[0],
        "start": int(parts[1]),
        "end": int(parts[2]),
        "peak_id": p
    })
df_peaks = pd.DataFrame(peak_data)

mapped_results = []

for chrom in df_final['chrom'].unique():
    

    subset_snps = df_final[df_final['chrom'] == chrom]
    subset_peaks = df_peaks[df_peaks['chrom'] == chrom]
    
    if subset_snps.empty or subset_peaks.empty: continue

    subset_snps = subset_snps.sort_values('pos')
    snp_pos = subset_snps['pos'].values
    peak_starts = subset_peaks['start'].values
    peak_ends = subset_peaks['end'].values
    peak_ids = subset_peaks['peak_id'].values
    

    left_idxs = np.searchsorted(snp_pos, peak_starts, side='left')
    right_idxs = np.searchsorted(snp_pos, peak_ends, side='right')
    
   
    for i in range(len(peak_ids)):
        l, r = left_idxs[i], right_idxs[i]
      
        if r > l:
            matched_rows = subset_snps.iloc[l:r].copy()
            matched_rows['Peak'] = peak_ids[i]
            mapped_results.append(matched_rows)


if len(mapped_results) > 0:
    df_mapped = pd.concat(mapped_results)
    df_export = df_mapped.rename(columns={"N_Cells": "N_Pos"})
    final_cols = ["SNP", "Peak", "Donor", "N_Pos","Count_Pos", "Count_Neg"]
    df_export = df_export[final_cols]
    
    print(f"Successfully mapped {len(df_export)} SNP-Peak pairs.")
    output_file = "asca_counts_input_cd4.csv"
    df_export.to_csv(output_file, sep="\t", index=False)
    print(f"Saved to {output_file}")
else:
    print("Warning: No SNPs fell within the provided Peaks.")

In [ ]:
#              SNP             Peak   Donor  N_Pos  N_Neg  Count_Pos  Count_Neg
#  chr1_10231_C_A  chr1:9908-10409  donor5     10      0          5          5
#  chr1_10291_C_T  chr1:9908-10409  donor4     10      0          0         11
#  chr1_10291_C_T  chr1:9908-10409  donor5     38      0          5         37
#  chr1_10327_T_C  chr1:9908-10409  donor4     12      0          3         10
#  chr1_10327_T_C  chr1:9908-10409  donor5     15      0          1         16

# CHECK FOR NEXT

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.io as sio


TARGET_CELL_TYPE = "cd4" 

target_barcodes = meta_df.index.values

print("Loading VCF...")
vcf_df = pd.read_csv("~/cellsnp_pre_sampler/merged_output/cellSNP.base.vcf.gz", 
                     sep="\t", comment='#', header=None, compression='gzip')
snp_ids = (vcf_df[0].astype(str) + "_" + vcf_df[1].astype(str) + "_" + 
           vcf_df[3].astype(str) + "_" + vcf_df[4].astype(str)).values


print("Loading Matrices...")
ad_mtx = sio.mmread("~/cellsnp_pre_sampler/merged_output/cellSNP.tag.AD.mtx").tocsr()
dp_mtx = sio.mmread("~/cellsnp_pre_sampler/merged_output/cellSNP.tag.DP.mtx").tocsr()


matrix_barcodes = pd.read_csv("~/cellsnp_pre_sampler/merged_output/cellSNP.samples.tsv", header=None)[0].values

print("Slicing matrix for target cells...")
_, matrix_idx, target_idx = np.intersect1d(matrix_barcodes, target_barcodes, return_indices=True)

if len(matrix_idx) == 0:
    raise ValueError("CHECK BARCODE FORMAT")


ad_mtx_sub = ad_mtx[:, matrix_idx]
dp_mtx_sub = dp_mtx[:, matrix_idx]


barcode_to_donor = meta['donor_id'].to_dict()
sub_donors = np.array([barcode_to_donor[bc] for bc in matrix_barcodes[matrix_idx]])

print(f"NUM.CELLS {ad_mtx_sub.shape[1]}")

results = []
unique_donors = np.unique(sub_donors)

for snp_idx in tqdm(range(ad_mtx_sub.shape[0])):
    row_dp = dp_mtx_sub[snp_idx, :].toarray().flatten()
    if row_dp.sum() == 0: continue
    
    row_ad = ad_mtx_sub[snp_idx, :].toarray().flatten()
    current_snp_id = snp_ids[snp_idx] 
    
    for donor in unique_donors:
        donor_mask = (sub_donors == donor)
        valid_mask = donor_mask & (row_dp > 0)
        
        if not np.any(valid_mask): continue
        
        count_pos = row_ad[valid_mask].sum()
        count_total = row_dp[valid_mask].sum()
        count_neg = count_total - count_pos
        n_cells = np.sum(valid_mask)
        
        if count_total < 10 or n_cells < 5: continue
        
        results.append({
            "SNP": current_snp_id,
            "CellType": TARGET_CELL_TYPE,
            "Donor": donor,
            "Count_Pos": count_pos,
            "Count_Neg": count_neg,
            "N_Cells": n_cells
        })

df_final = pd.DataFrame(results)
print(df_final.head())

#SAVE
# df_final.to_csv(...)